In [1]:
import numpy as np
from obj_func import obj_func
from read_instance import read_instance
import random
import seaborn as sns
from random import randint
import sys 

In [476]:
#file=open(r"E:\monit\Documents\TTP\Instancias\a280-ttp\a280_n279_bounded-strongly-corr_01.ttp","r")
file=open(r"E:\monit\Documents\TTP\Instancias\a280-ttp\a280_n279_uncorr_01.ttp","r")
#file=open(r"E:\monit\Documents\TTP\Instancias\a280-ttp\my_test.ttp","r")
dic=read_instance(file)


In [477]:
def create_knap_pop(dic,c):
    pop_knap=np.zeros((c,dic["number of items"]))
    for i in range(pop_knap.shape[0]):
        item_pick=[]
        for x in range(dic["number of items"]): 
            item_pick.append(random.randint(0, 1))
        item_pick2=np.array(item_pick)
        pop_knap[i]=item_pick2
    return pop_knap

In [478]:
def create_distance_matrix(dic):
    chords=dic["coords citites"]
    matrix=np.zeros((len(chords),len(chords)))
    ind1=0
    for x in chords:
        ind2=0
        for f in chords:
            point1=np.array((int(x[0]),int(x[1])))
            point2=np.array((int(f[0]),int(f[1])))
            matrix[ind1,ind2]=np.linalg.norm(point1-point2)
            ind2=ind2+1
        ind1=ind1+1
    return matrix

In [479]:
def create_tsp_pop(dic,x,matrix):
    pop_tsp=np.zeros((x,dic["number cities"]-1))
    minimo=np.amin(matrix[matrix>0])
    maximo=np.amax(matrix[matrix>0])
    #print(minimo,maximo,pop_tsp.shape[1])
    for i in range(pop_tsp.shape[0]):
        #order_cities=list(range(2,dic["number cities"]+1 ))
        dist=(maximo-minimo) * np.random.random_sample((pop_tsp.shape[1] )) +minimo    
        pop_tsp[i,:]=dist
    return pop_tsp

In [496]:
def cont_to_disc_tsp(dic,pop_tsp,matrix):
    pop_tsp_disc=np.zeros((pop_tsp.shape))
    matrix[:,0]=0
 
    for x in range(pop_tsp.shape[0]):
        line_check=matrix[0,:].copy()
        orden=[]
        for i in pop_tsp[x,:]:
            #print(i,"i")
            diff=abs(line_check-i)
            pick_list=list(zip(*np.where(diff==np.amin(diff))))
            pick= int(random.choice(pick_list)[0])
            #pick=int(pick_list[0][0])
            orden.append(pick+1)
            line_check=matrix[pick,:].copy()
            for l in orden:
                line_check[l-1]=0
            #print(diff,"diff")
            #print(pick,'city')
            #print(line_check,"newline'")
        #print(orden,"orden_ciudades",len(set(orden)),"unicos",len(orden),'largo del orden')
        pop_tsp_disc[x,:]=orden
    return pop_tsp_disc


In [481]:
def fit_loop(dic,pop_tsp,pop_knap):
    fitness=np.zeros((pop_tsp.shape[0]))
    for l in range(pop_tsp.shape[0]):
        fitness[l]=obj_func(dic,pop_tsp[l,:],pop_knap[l,:])
    return fitness

In [503]:

def diff_evolution(fitness,pop_tsp,pop_knap,f,r,matrix,dic):
    dim=np.shape(pop_tsp)[0]
    minimo=np.amin(matrix[matrix>0])
    maximo=np.amax(matrix[matrix>0])
    new_population_tsp=np.zeros(np.shape(pop_tsp))
    new_population_knap=np.zeros(np.shape(pop_knap))
    for k in range(dim): 
   
        selector=random.sample(range(dim), 3)
        #print(selector)
        #diff evolution for tsp
        #print(pop_tsp[selector[2]],pop_tsp[selector[0]],selector)
        mutated_tsp=pop_tsp[selector[2]].copy()  + (f*(pop_tsp[selector[0]].copy()-pop_tsp[selector[1]].copy()))
        mutated_tsp=np.reshape(np.clip(mutated_tsp,minimo,maximo),(1,mutated_tsp.shape[0]))
        pre_pop_tsp=np.zeros((mutated_tsp.shape))
        for z in range(pop_tsp.shape[1]):
            if random.uniform(0,1)>r:
                pre_pop_tsp[0,z]=mutated_tsp[0,z]
            else:
                pre_pop_tsp[0,z]=pop_tsp[selector[2]].copy()[z]
        pre_pop_tsp_fit=(cont_to_disc_tsp(dic,pre_pop_tsp,matrix)).astype(np.int32)
        #print(pre_pop_tsp_fit,cont_to_disc_tsp(dic,np.reshape(pop_tsp[k,:],(1,pop_tsp.shape[1])),matrix))
        
        #diff evolution for knap, bin dif evolutio
        pre_nap=pop_knap[selector[2],:].copy()
        #print(pop_knap[selector[0],:],"before")
        #print(pop_knap[selector[1],:],pop_knap[selector[2],:],"bins")
        for l in range(pop_knap.shape[1]):
            if pop_knap[selector[0],l]!=pop_knap[selector[1],l] and random.uniform(0,1)>f and random.uniform(0,1)>r:
                pre_nap[l]=1-pre_nap[l]
        #print(pre_nap,'after')
        fitness1=fitness[k]
        fitness2=obj_func(dic,pre_pop_tsp_fit[0,:],pre_nap)            
        if fitness2>fitness1:
            new_population_tsp[k,:]=np.reshape(pre_pop_tsp,(pre_pop_tsp.shape[1]))
            new_population_knap[k,:]=pre_nap
        else:
            new_population_tsp[k,:]=pop_tsp[k,:]
            new_population_knap[k,:]=pop_knap[k,:]
        #print(fitness1,fitness2)
    
     
    return new_population_tsp,new_population_knap
    
    

In [508]:
ps=50#pop length
iterations=400
cr=0.7
f=0.5
matrix=create_distance_matrix(dic)
pop_tsp=create_tsp_pop(dic,ps,matrix)
pop_knap=create_knap_pop(dic,ps)
pop_tsp_for_fit=cont_to_disc_tsp(dic,pop_tsp,matrix)
fitness=fit_loop(dic,pop_tsp_for_fit,pop_knap)
z=True
l=0
lastfit=np.amax(fitness)
count=0
while z :
#for l in range(iterations):
    pop_tsp,pop_knap=diff_evolution(fitness,pop_tsp,pop_knap,f,cr,matrix,dic)
    pop_tsp_for_fit=cont_to_disc_tsp(dic,pop_tsp,matrix)
    fitness=fit_loop(dic,pop_tsp_for_fit,pop_knap)
    print("generation: ",l, " maximo: ", np.amax(fitness))  
    l=l+1
    
    if lastfit>=np.amax(fitness):
        count=count+1
    else:
        count=0
        lastfit=np.amax(fitness)
    if count==100:
        z=False
    
p1 = sns.heatmap(np.reshape(fitness,(1,fitness.shape[0])))    

generation:  0  maximo:  -1628379.9718193854
generation:  1  maximo:  -1543155.8096638168


KeyboardInterrupt: 